In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [2]:
df = load_iris()
X = df['data']
y = df['target']

In [3]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=50)

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

#RBF network class

In [5]:
class RBFNetwork:
  def __init__(self,n_hidden_neurons=10):
    self.n_hidden_neurons = n_hidden_neurons
    self.centers = None
    self.spread = None
    self.weights = None
#Gaussian RBF
  def _gaussian_rbf(self,X,center, spread):
    dist = np.linalg.norm(X - center,axis=1,ord=2)
    return np.exp(-dist/(2*spread**2))

#Hidden Layer Activation: Compute the RBF output matrix G
  def _compute_activation_matrix(self,X):
    G = np.zeros((X.shape[0], self.n_hidden_neurons))
    for i in range(self.n_hidden_neurons):
      G[:,i] = self._gaussian_rbf(X,self.centers[i],self.spread)
    return G
  #Training Method (2 stages)
  def fit(self,X,y):
    #1. Unsupervised Stage : Determining Centers (using K-Means clustering)
    kmeans = KMeans(n_clusters=self.n_hidden_neurons, random_state = 42, n_init='auto')
    kmeans.fit(X)
    self.centers = kmeans.cluster_centers_

    #Determine the spread: Max distance between centers
    dmax  = 0
    for i in range(self.n_hidden_neurons):
      for j in range(i+1, self.n_hidden_neurons):
        d = np.linalg.norm(self.centers[i] - self.centers[j])
        if d > dmax:
          dmax = d

    self.spread = dmax/np.sqrt(2*self.n_hidden_neurons)

    #2. Supervised Stage: Find weights

    G = self._compute_activation_matrix(X)

    #Convert target labels to one-hot encoding for multi-class classification
    n_classes = len(np.unique(y))
    y_one_hot = np.eye(n_classes)[y]

    #Find optimal weights (W)
    G_pseudo_inverse = np.linalg.pinv(G)
    self.weights = np.dot(G_pseudo_inverse,y_one_hot)
#prediction
  def predict(self,X):
    #Activation Matrix G for new data
    G = self._compute_activation_matrix(X)

    #Compute the final output scores
    output_scores = np.dot(G,self.weights)

    return np.argmax(output_scores,axis=1)

# training and evaluating the model

In [6]:
rbf_net = RBFNetwork(n_hidden_neurons=20)
rbf_net.fit(X_train_scaled,y_train)

In [7]:
y_pred = rbf_net.predict(X_test_scaled)

In [8]:
accuracy_score(y_test,y_pred)

0.9333333333333333

# Digits

In [9]:
df = load_digits()
X = df['data']
y = df['target']

In [10]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [11]:
y

array([0, 1, 2, ..., 8, 9, 8])